# Exploring the PLAsTiCC competition

First step in a kaggle competition dataset

https://www.kaggle.com/c/PLAsTiCC-2018/

1. File Descriptions

    * data_note: Start here for a more extensive context than we can provide in this space. You'll get helpful background on both this dataset and the relevant astronomy. This is also available on arxiv.org. We've also posted kernels with additional context about the relevant astronomy and the approaches astronomers have taken in the past.
    * [training/test]_set_metadata: Information about objects in the that doesn't change over time, like the coordinates of the object.
    * [training/test]_set: Time series of observations of the objects. Maps to the metadata via object_id.
    * test_set_sample: The first million rows of the test set.
    * sample_submission: A valid submission file. You can submit probabilities of precision float64, but we highly recommend sticking to float16 for your initial iterations to minimize upload time.

2. Metadata File Column Descriptions

    * object_id: unique object identifier. Int32
    * ra: right ascension, sky coordinate: co-longitude in degrees. Float32
    * decl: declination, sky coordinate: co-latitude in degrees. Float32
    * gal_l: galactic longitude in degrees. Float32
    * gal_b: galactic latitude in degrees. Float32
    * ddf: A flag to identify the object as coming from the DDF survey area (with value DDF = 1 for the DDF, DDF = 0 for the WFD survey). Note that while the DDF fields are contained within the full WFD survey area, the DDF fluxes have significantly smaller uncertainties. Boolean
    * hostgal_specz: the spectroscopic redshift of the source. This is an extremely accurate measure of redshift, available for the training set and a small fraction of the test set. Float32
    * hostgal_photoz: The photometric redshift of the host galaxy of the astronomical source. While this is meant to be a proxy for hostgal_specz, there can be large differences between the two and should be regarded as a far less accurate version of hostgal_specz. Float32
    * hostgal_photoz_err: The uncertainty on the hostgal_photoz based on LSST survey projections. Float32
    * distmod: The distance to the source calculated from hostgal_photoz and using general relativity. Float32
    * mwebv: MW E(B-V). this ‘extinction’ of light is a property of the Milky Way (MW) dust along the line of sight to the astronomical source, and is thus a function of the sky coordinates of the source ra, decl. This is used to determine a passband dependent dimming and redenning of light from astronomical sources as described in subsection 2.1, and based on the Schlafly et al. (2011) and Schlegel et al. (1998) dust models. Float32
    * target: The class of the astronomical source. This is provided in the training data. Correctly determining the target (correctly assigning classification probabilities to the objects) is the ‘goal’ of the classification challenge for the test data. Note that there is one class in the test set that does not occur in the training set: class_99 serves as an "other" class for objects that don't belong in any of the 14 classes in the training set. Int8

3. Data File Column Descriptions

    * object_id: Foreign key with the metadata. Int32
    * mjd: the time in Modified Julian Date (MJD) of the observation. Can be read as days since November 17, 1858. Can be converted to Unix epoch time with the formula unix_time = (MJD−40587)×86400. Float64
    * passband: The specific LSST passband integer, such that u, g, r, i, z, Y = 0, 1, 2, 3, 4, 5 in which it was viewed. Int8
    * flux: the measured flux (brightness) in the passband of observation as listed in the passband column. These values have already been corrected for dust extinction (mwebv), though heavily extincted objects will have larger uncertainties (flux_err) in spite of the correction. Float32
    * flux_err: the uncertainty on the measurement of the flux listed above. Float32
    * detected: If 1, the object's brightness is significantly different at the 3-sigma level relative to the reference template. Only objects with at least 2 detections are included in the dataset. Boolean

4. Caveats: A few caveats about the light-curve data are as follows:

    * Data gaps: Different passbands are taken at different times, sometimes many days apart.
    * Galactic vs extragalactic: The given redshift for objects in our own galaxy is given as zero.
    * Negative Flux: Due to statistical fluctuations (of e.g. the sky brightness) and the way the brightness is estimated, the flux may be negative for dim sources, where the true flux is close to zero. Second, if the pre-survey image actually contains a flux brighter than its true ‘zero’, this can lead to a negative flux when the difference is computed.


**Loss function**

Submissions are evaluated using a weighted multi-class logarithmic loss. The overall effect is such that each class is roughly equally important for the final score.

Each object has been labeled with one type. For each object, you must submit a set of predicted probabilities (one for every category). The formula is then:

```
Log Loss=−⎛⎝∑Mi=1wi⋅∑Nij=1yijNi⋅lnpij∑Mi=1wi⎞⎠
```

where: 
1. N is the number of objects in the class set,
2. M is the number of classes,  
3. ln is the natural logarithm, 
4. yij is 1 if observation (i) belongs to class (j) and 0 otherwise, 
5. pij is the predicted probability that observation i belongs to class j .

The submitted probabilities for a given object are not required to sum to one because they are rescaled prior to being scored (each row is divided by the row sum). In order to avoid the extremes of the log function, predicted probabilities are replaced with max(min(p,1−10^−15),10^−15) .

**Submission File**

For each object ID in the test set, you must predict a probability for each of the different possible classes. The file should contain a header and have the following format:

```
object_id,class_6,class_15,class_16,class_42,class_52,class_53,class_62,class_64,class_65,class_67,class_88,class_90,class_92,class_95,class_99
13,0,0.1,0,0.1,0,0.3,0,0,0,0,0,0.5,0,0,0
14,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
17,0.75,0.23,0,0,0.01,0,0,0,0,0.01,0,0,0,0,0
etc.
```

**Starter kit of the organizers:**

https://www.kaggle.com/michaelapers/the-plasticc-astronomy-starter-kit

**Full archive description**

https://arxiv.org/pdf/1810.00001.pdf

For a PLAsTiCCentry to be valid, it will have to include probabilities for each class andastronomical source, i.e. an entry cannot leave out probabilities on any source, or class. To win the challenge, all entries should be valid and minimize the PLAsTiCC metric score. The metric used for the challenge is a weighted log-loss metric:

```
L= −∑^{M}_{j= 1} wj · [ ∑^{N}_{i=1} (1 / Nj) τ_{i,j} ln(Pij) / ∑^{M}_{j=1} wj ] ,(1)
```

where

`τ_{i,j}= 1` if the `i`th object comes from the `j`th class and `0` otherwise, 

and `Nj` is the number of objects in any given class `j`, 

and `wj` are individual weights per class which reflect relative contribution to the overall metric (depending on e.g. how desireable it is to have objects in class `j` classified correctly). 

**These wj are hidden to the participants of the challenge.** The averaging over Nj reflects that PLAsTiCC is designed to reward those who classify all objects well on average, rather than focusing on one particular class. This is discussed more in an accompanying paper (see Malz,  Hloˇzeket al.2018 which isreleased with this note).

## Setings
### Notebook configuration

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

### Packages

In [2]:
import os  # manage path and os interaction
import numpy as np  # manage array and math operation
import pandas as pd  # manage table

import matplotlib as mpl  # plot data
import matplotlib.pyplot as plt  # plot data

import pandas_profiling

In [3]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier

## Import data

In [4]:
data_path = '../data/PLAsTiCC-2018/'
df_set = pd.read_csv(data_path + 'training_set.csv.zip', compression='zip')
df_meta = pd.read_csv(data_path + 'training_set_metadata.csv')

In [7]:
profile_set = pandas_profiling.ProfileReport(df_set)
profile_set.to_file(output_file="plasticc_set_pandas_profile.html")

In [8]:
profile_meta = pandas_profiling.ProfileReport(df_meta)
profile_meta.to_file(output_file="plasticc_meta_pandas_profile.html")

## First classification use meta data

## Simple aggregation of spectral information

## Beyond

# End of script